# Software Environment and Cloud Computing

[![Virus on Linux](fig/virus.png)](https://www.gnu.org/fun/jokes/evilmalware.en.html)

In modern computational science, running code is only half the
challenge.
The other half is managing the software environment it depends on.
Different projects may require different compilers, libraries, or
Python packages, and the ability to reproduce results across HPC
systems, laptops, and cloud platforms is important.

This week, we will learn practical tools for creating and managing
software environments:
* Package managers (e.g., `apt-get`) for installing software at the
  system level
* Building from source when precompiled packages are unavailable or
  when you do not have root access to the system.
* HPC `module` for loading different versions of software on a shared
  system.
* Python virtual environments for dependency isolation
* Containers (i.e., Docker) for reproducibility and portability

[![Dependency](fig/dependency.png)](https://xkcd.com/2347/)

[![sudo](fig/sandwich.png)](https://xkcd.com/149/)

[![Python Environment](fig/python_environment.png)](https://xkcd.com/1987/)